In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from io import StringIO
import boto3
import s3fs
import awscli
import timeit

In [2]:
tic=timeit.default_timer()
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

4.909999999824777e-05

In [3]:
s3 = boto3.resource('s3')

for bucket in s3.buckets.all():
    print(bucket.name)

team3-final-bucket
wshih-bucket


In [4]:
my_bucket = s3.Bucket('team3-final-bucket')
for my_bucket_object in my_bucket.objects.all():
    print(my_bucket_object.key)

Resources_clean/
Resources_clean/acquisitions_clean.csv
Resources_clean/degrees_clean.csv
Resources_clean/funding_rounds_clean.csv
Resources_clean/funds_clean.csv
Resources_clean/investments_clean.csv
Resources_clean/ipo_clean.csv
Resources_clean/milestones_clean.csv
Resources_clean/objects_clean.csv
Resources_clean/offices_clean.csv
Resources_clean/people_clean.csv
Resources_clean/relationships_clean.csv
Resources_raw/
Resources_raw/acquisitions.csv
Resources_raw/degrees.csv
Resources_raw/funding_rounds.csv
Resources_raw/funds.csv
Resources_raw/investments.csv
Resources_raw/ipos.csv
Resources_raw/milestones.csv
Resources_raw/objects.csv
Resources_raw/offices.csv
Resources_raw/people.csv
Resources_raw/relationships.csv
sql_db/
sql_db/acquisitions_clean.csv
sql_db/degrees_clean.csv
sql_db/funding_round_filled.csv
sql_db/funds_clean.csv
sql_db/investments_cleaner.csv
sql_db/ipos_cleaner.csv
sql_db/milestones_cleaner.csv
sql_db/objects_clean.csv
sql_db/offices_clean.csv
sql_db/people_clea

In [6]:
client = boto3.client('s3')
path = 's3://team3-final-bucket/sql_db/'

In [17]:
fund_rounds_df= pd.read_csv((path + 'funding_round_filled.csv'), index_col=False)
fund_rounds_df

,Unnamed: 0,funding_round_id,object_id,funded_at,funding_round_type,funding_round_code,raised_amount_usd,raised_amount,raised_currency_code,pre_money_valuation_usd,pre_money_valuation,pre_money_currency_code,post_money_valuation_usd,post_money_valuation,post_money_currency_code,participants,is_first_round,is_last_round
0,0,1,c:4,12/1/2006,series-b,b,8500000,8500000,USD,0,0,0,0,0,0,2,0,0
1,1,2,c:5,9/1/2004,angel,angel,500000,500000,USD,0,0,USD,0,0,USD,2,0,1
2,2,3,c:5,5/1/2005,series-a,a,12700000,12700000,USD,115000000,115000000,USD,0,0,USD,3,0,0
3,3,4,c:5,4/1/2006,series-b,b,27500000,27500000,USD,525000000,525000000,USD,0,0,USD,4,0,0
4,4,5,c:7299,5/1/2006,series-b,b,10500000,10500000,USD,0,0,0,0,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52621,52923,57948,c:211890,12/12/2013,series-a,a,3000000,3000000,USD,0,0,USD,0,0,USD,1,1,1
52622,52924,57949,c:267427,2/6/2010,venture,partial,570000,570000,USD,0,0,USD,0,0,USD,0,0,1
52623,52925,57950,c:261728,2/6/2010,venture,unattributed,2184100,2184100,USD,0,0,USD,0,0,USD,0,0,1
52624,52926,57951,c:285864,12/12/2013,series-a,a,790783,790783,USD,0,0,USD,0,0,USD,0,1,1


In [18]:
fund_rounds_df.columns.to_list()

['Unnamed: 0',
 'funding_round_id',
 'object_id',
 'funded_at',
 'funding_round_type',
 'funding_round_code',
 'raised_amount_usd',
 'raised_amount',
 'raised_currency_code',
 'pre_money_valuation_usd',
 'pre_money_valuation',
 'pre_money_currency_code',
 'post_money_valuation_usd',
 'post_money_valuation',
 'post_money_currency_code',
 'participants',
 'is_first_round',
 'is_last_round']

In [26]:
# Dropping redundant columns
funds_df = fund_rounds_df[['object_id', 'funded_at', 'funding_round_type', 'participants']]
funds_df

,object_id,funded_at,funding_round_type,participants
0,c:4,12/1/2006,series-b,2
1,c:5,9/1/2004,angel,2
2,c:5,5/1/2005,series-a,3
3,c:5,4/1/2006,series-b,4
4,c:7299,5/1/2006,series-b,2
...,...,...,...,...
52621,c:211890,12/12/2013,series-a,1
52622,c:267427,2/6/2010,venture,0
52623,c:261728,2/6/2010,venture,0
52624,c:285864,12/12/2013,series-a,0


In [34]:
# Converting funded_at column to date type
funds_df['funded_at'] = pd.to_datetime(funds_df.funded_at)
funds_df

C:\Users\hp\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,object_id,funded_at,funding_round_type,participants
0,c:4,2006-12-01,series-b,2
1,c:5,2004-09-01,angel,2
2,c:5,2005-05-01,series-a,3
3,c:5,2006-04-01,series-b,4
4,c:7299,2006-05-01,series-b,2
...,...,...,...,...
52621,c:211890,2013-12-12,series-a,1
52622,c:267427,2010-02-06,venture,0
52623,c:261728,2010-02-06,venture,0
52624,c:285864,2013-12-12,series-a,0


In [74]:
funds_df.dtypes

object_id                     object
funded_at             datetime64[ns]
funding_round_type            object
participants                   int64
dtype: object

In [76]:
# Checking null values
for column in funds_df.columns:
    print(f"Column {column} has {funds_df[column].isnull().sum()} null values")

Column object_id has 0 null values
Column funded_at has 0 null values
Column funding_round_type has 0 null values
Column participants has 0 null values


In [41]:
# There are missing values in the funded_at column

In [21]:
funds_df[funds_df['funding_round_type'].isnull()]

,object_id,funding_round_type,participants


In [22]:
funds_df[funds_df['participants'].isnull()]

,object_id,funding_round_type,participants


As we cannot define the latest funding round by its funding type we will focus on dates. But because there are missing dates in the column we will also need to sort funding types as well only for those objects. Thus: 

In [42]:
# Sorting object_ids, then funded_at and then funding_round_types
funds_sorted = funds_df.sort_values(['object_id', 'funded_at', 'funding_round_type'], ascending=[True, True, True])
funds_sorted

,object_id,funded_at,funding_round_type,participants
787,c:1,2005-10-01,series-a,2
788,c:1,2007-01-01,series-b,3
2038,c:1,2008-05-19,series-c+,4
1454,c:1001,2008-02-26,series-a,3
5937,c:10014,2008-09-01,angel,1
...,...,...,...,...
5104,c:9989,2009-02-01,angel,0
2877,c:9994,2007-01-01,angel,1
5421,c:9994,2008-05-01,angel,1
2886,c:9995,2008-08-25,angel,0


In [25]:
funds_sorted['funding_round_type'].unique()

array(['series-a', 'series-b', 'series-c+', 'angel', 'venture', 'other',
       'private-equity', 'post-ipo', 'crowdfunding'], dtype=object)

Finding the lates funding roud type in the table below that can be used as a feature in ML algorithms:

In [45]:
# Keeping only last funding round of each unique object
last_funding = funds_sorted.drop_duplicates(subset=['object_id'], keep='last')
last_funding

,object_id,funded_at,funding_round_type,participants
2038,c:1,2008-05-19,series-c+,4
1454,c:1001,2008-02-26,series-a,3
5937,c:10014,2008-09-01,angel,1
41069,c:10015,2013-08-13,series-c+,5
25297,c:100155,2012-01-26,series-b,2
...,...,...,...,...
22952,c:99853,2011-09-06,angel,0
5104,c:9989,2009-02-01,angel,0
5421,c:9994,2008-05-01,angel,1
2886,c:9995,2008-08-25,angel,0


In [13]:
fund_rounds_df.dtypes

Unnamed: 0                   int64
funding_round_id             int64
object_id                   object
funded_at                   object
funding_round_type          object
funding_round_code          object
raised_amount_usd            int64
raised_amount                int64
raised_currency_code        object
pre_money_valuation_usd      int64
pre_money_valuation          int64
pre_money_currency_code     object
post_money_valuation_usd     int64
post_money_valuation         int64
post_money_currency_code    object
participants                 int64
is_first_round               int64
is_last_round                int64
dtype: object

Now we need to find the number of rounds and the number of participants

In [59]:
funds_agg = funds_sorted.groupby(['object_id']).agg({'funding_round_type':'count', 'participants':'sum'})
funds_agg

,funding_round_type,participants
object_id,,
c:1,3,9
c:1001,1,3
c:10014,1,1
c:10015,5,19
c:100155,3,5
...,...,...
c:99853,1,0
c:9989,2,0
c:9994,2,2


In [58]:
last_funding_clean = last_funding[['object_id','funding_round_type']]
last_funding_clean = last_funding_clean.rename(columns={"funding_round_type":"last_funding_round_type"})
last_funding_clean

,object_id,last_funding_round_type
2038,c:1,series-c+
1454,c:1001,series-a
5937,c:10014,angel
41069,c:10015,series-c+
25297,c:100155,series-b
...,...,...
22952,c:99853,angel
5104,c:9989,angel
5421,c:9994,angel
2886,c:9995,angel


In [61]:
funding_rounds_final = pd.merge(last_funding_clean, funds_agg, how='left', on=['object_id'])
funding_rounds_final = funding_rounds_final.rename(columns={"funding_round_type":"funding_rounds_count", "participants":"number_of_participants"})
funding_rounds_final

,object_id,last_funding_round_type,funding_rounds_count,number_of_participants
0,c:1,series-c+,3,9
1,c:1001,series-a,1,3
2,c:10014,angel,1,1
3,c:10015,series-c+,5,19
4,c:100155,series-b,3,5
...,...,...,...,...
31702,c:99853,angel,1,0
31703,c:9989,angel,2,0
31704,c:9994,angel,2,2
31705,c:9995,angel,1,0


In [102]:
# Loading companies from my file
companies_df = pd.read_csv('companies_info.csv')
companies = companies_df[['object_id','investment_rounds','funding_rounds']]
companies

,object_id,investment_rounds,funding_rounds
0,c:1,0,3
1,c:10,0,0
2,c:100,0,0
3,c:10000,0,0
4,c:10001,0,0
...,...,...,...
196548,c:99940,0,0
196549,c:9995,0,1
196550,c:9996,0,0
196551,c:9997,0,0


In [103]:
sum_col = companies['investment_rounds'] + companies['funding_rounds']

In [104]:
companies['rounds_total'] = sum_col

C:\Users\hp\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [105]:
companies

,object_id,investment_rounds,funding_rounds,rounds_total
0,c:1,0,3,3
1,c:10,0,0,0
2,c:100,0,0,0
3,c:10000,0,0,0
4,c:10001,0,0,0
...,...,...,...,...
196548,c:99940,0,0,0
196549,c:9995,0,1,1
196550,c:9996,0,0,0
196551,c:9997,0,0,0


In [106]:
# Merging companies and funding rounds
companies_funding_rounds_df = pd.merge(companies, funding_rounds_final, how='left', on=['object_id'])
companies_funding_rounds_df

,object_id,investment_rounds,funding_rounds,rounds_total,last_funding_round_type,funding_rounds_count,number_of_participants
0,c:1,0,3,3,series-c+,3.0,9.0
1,c:10,0,0,0,NaN,NaN,NaN
2,c:100,0,0,0,NaN,NaN,NaN
3,c:10000,0,0,0,NaN,NaN,NaN
4,c:10001,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
196548,c:99940,0,0,0,NaN,NaN,NaN
196549,c:9995,0,1,1,angel,1.0,0.0
196550,c:9996,0,0,0,NaN,NaN,NaN
196551,c:9997,0,0,0,NaN,NaN,NaN


In [107]:
companies_funding_rounds_df[companies_funding_rounds_df['rounds_total'] != 0]

,object_id,investment_rounds,funding_rounds,rounds_total,last_funding_round_type,funding_rounds_count,number_of_participants
0,c:1,0,3,3,series-c+,3.0,9.0
13,c:1001,0,1,1,series-a,1.0,3.0
18,c:10014,0,1,1,angel,1.0,1.0
19,c:10015,0,5,5,series-c+,5.0,19.0
20,c:100155,0,3,3,series-b,3.0,5.0
...,...,...,...,...,...,...,...
196537,c:99853,0,1,1,angel,1.0,0.0
196540,c:9989,0,2,2,angel,2.0,0.0
196547,c:9994,0,2,2,angel,2.0,2.0
196549,c:9995,0,1,1,angel,1.0,0.0


In [108]:
companies_funding_rounds_df[companies_funding_rounds_df['funding_rounds_count'].notnull()]

,object_id,investment_rounds,funding_rounds,rounds_total,last_funding_round_type,funding_rounds_count,number_of_participants
0,c:1,0,3,3,series-c+,3.0,9.0
13,c:1001,0,1,1,series-a,1.0,3.0
18,c:10014,0,1,1,angel,1.0,1.0
19,c:10015,0,5,5,series-c+,5.0,19.0
20,c:100155,0,3,3,series-b,3.0,5.0
...,...,...,...,...,...,...,...
196537,c:99853,0,1,1,angel,1.0,0.0
196540,c:9989,0,2,2,angel,2.0,0.0
196547,c:9994,0,2,2,angel,2.0,2.0
196549,c:9995,0,1,1,angel,1.0,0.0


In [109]:
# Combining rounds:
def fill_rounds(a,b):
    if a == 0:
        return b
    else:
        return a

In [110]:
companies_funding_rounds_df['rounds_total'] = companies_funding_rounds_df.apply(lambda row : fill_rounds(row['rounds_total'], row['funding_rounds_count']), axis = 1)

In [111]:
companies_funding_rounds_df

,object_id,investment_rounds,funding_rounds,rounds_total,last_funding_round_type,funding_rounds_count,number_of_participants
0,c:1,0,3,3.0,series-c+,3.0,9.0
1,c:10,0,0,NaN,NaN,NaN,NaN
2,c:100,0,0,NaN,NaN,NaN,NaN
3,c:10000,0,0,NaN,NaN,NaN,NaN
4,c:10001,0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
196548,c:99940,0,0,NaN,NaN,NaN,NaN
196549,c:9995,0,1,1.0,angel,1.0,0.0
196550,c:9996,0,0,NaN,NaN,NaN,NaN
196551,c:9997,0,0,NaN,NaN,NaN,NaN


In [112]:
companies_funding_rounds_df[companies_funding_rounds_df['rounds_total'].notnull()]

,object_id,investment_rounds,funding_rounds,rounds_total,last_funding_round_type,funding_rounds_count,number_of_participants
0,c:1,0,3,3.0,series-c+,3.0,9.0
13,c:1001,0,1,1.0,series-a,1.0,3.0
18,c:10014,0,1,1.0,angel,1.0,1.0
19,c:10015,0,5,5.0,series-c+,5.0,19.0
20,c:100155,0,3,3.0,series-b,3.0,5.0
...,...,...,...,...,...,...,...
196537,c:99853,0,1,1.0,angel,1.0,0.0
196540,c:9989,0,2,2.0,angel,2.0,0.0
196547,c:9994,0,2,2.0,angel,2.0,2.0
196549,c:9995,0,1,1.0,angel,1.0,0.0


In [113]:
companies_funding_rounds_df.columns.to_list()

['object_id',
 'investment_rounds',
 'funding_rounds',
 'rounds_total',
 'last_funding_round_type',
 'funding_rounds_count',
 'number_of_participants']

In [114]:
companies_funding_rounds = companies_funding_rounds_df[['object_id',
 'rounds_total',
 'last_funding_round_type',
 'number_of_participants']]
companies_funding_rounds

,object_id,rounds_total,last_funding_round_type,number_of_participants
0,c:1,3.0,series-c+,9.0
1,c:10,NaN,NaN,NaN
2,c:100,NaN,NaN,NaN
3,c:10000,NaN,NaN,NaN
4,c:10001,NaN,NaN,NaN
...,...,...,...,...
196548,c:99940,NaN,NaN,NaN
196549,c:9995,1.0,angel,0.0
196550,c:9996,NaN,NaN,NaN
196551,c:9997,NaN,NaN,NaN


In [115]:
companies_funding_rounds.to_csv('companies_funding_rounds.csv', sep=',', index=False)

In [68]:
# bucket = 'Resources_clean'
# csv_buffer = StringIO()
# funding_rounds_final.to_csv(csv_buffer)
# s3.Object('Resources_clean', 'funding_rounds_final').put(Body=csv_buffer.getvalue())

In [70]:
bucket = 'team3-final-bucket'
csv_buffer = StringIO()
funding_rounds_final.to_csv(csv_buffer)
content = csv_buffer.getvalue()
def to_s3(bucket,'funding_rounds_final.csv', content):
    client = boto3.client('s3')
    k = "Resources_clean/"+funding_rounds_final.csv
    client.put_object(Bucket=bucket, Key=k, Body=content)

SyntaxError: invalid syntax (<ipython-input-70-8dfac47ae89d>, line 5)

In [72]:
funding_rounds_final.to_csv('funding_rounds_final.csv', sep=',', index=False)